In [77]:
import pandas as pd

In [78]:
products = pd.read_csv("products.csv")
transactions = pd.read_csv("transactions.csv")

Для пользователя - [id_пользователя, кол-во купленных продуктов, показатель перезаказов]
Показатель перезаказов - это характеристика, отражающая процент продуктов, которые пользователь покупал больше 1 раза

In [79]:
users_embeddings = transactions.groupby("user_id").agg({"product_id": "count", "reordered": "sum"}).rename(columns={"product_id": "n_bought"}).reset_index()

In [80]:
users_embeddings.reordered = users_embeddings.reordered / users_embeddings.n_bought

Для продуктов - [id продукта, процент перезаказа этого продукта среди всех пользователей, кол-во заказов данного продукта всеми пользователями, процент заказа данного продукта конкретным пользователем]

In [81]:
products_embeddings = transactions.groupby("product_id").agg({"user_id": "count", "reordered": "sum"})

In [82]:
products_embeddings["reordered"] = products_embeddings.reordered / products_embeddings.user_id

In [83]:
products_embeddings.columns = ["n_ordered", "reorder_coeff"]

In [87]:
products_embeddings.loc[138]

n_ordered        260.000000
reorder_coeff      0.392308
Name: 138, dtype: float64

In [88]:
products_embeddings.reset_index(inplace=True)

процент заказа данного продукта конкретным пользователем

In [89]:
product_user_coeff = transactions.groupby(["user_id", "product_id"]).agg({"reordered":"count"})

In [41]:
product_user_coeff = product_user_coeff.reset_index().rename(columns={"reordered": "user_ordered"})

In [42]:
product_user_coeff = pd.merge(products_embeddings[["product_id", "n_ordered"]], product_user_coeff, on="product_id")

In [43]:
product_user_coeff["product_user_coeff"] = product_user_coeff.user_ordered / product_user_coeff.n_ordered

In [44]:
product_user_coeff = product_user_coeff[["product_id", "user_id", "product_user_coeff"]]

In [45]:
users_embeddings.columns = ["user_id", "user_total_bought", "user_reorder_coeff"]

In [46]:
products_embeddings.columns = ["product_id", "product_total_ordered", "product_reorder_coeff"]

In [47]:
product_user_coeff = pd.merge(product_user_coeff, users_embeddings, on="user_id")

In [48]:
product_user_coeff = pd.merge(product_user_coeff, products_embeddings, on="product_id")

Для "игреков" можно например взять 1, если продукт был перезаказан, 0 , если не покупался/не перезаказывался 

In [49]:
y = transactions.groupby(["product_id", "user_id"])["reordered"].agg("max").reset_index()

In [50]:
y.columns = ["product_id", "user_id", "reordered_target"]

In [51]:
df = pd.merge(product_user_coeff, y, on=["product_id", "user_id"])

In [52]:
df.head()

,product_id,user_id,product_user_coeff,user_total_bought,user_reorder_coeff,product_total_ordered,product_reorder_coeff,reordered_target
0,1,138,0.001291,148,0.628378,1549,0.678502,1.0
1,1,777,0.000646,113,0.734513,1549,0.678502,0.0
2,1,1052,0.001291,48,0.500000,1549,0.678502,1.0
3,1,1480,0.001937,192,0.734375,1549,0.678502,1.0
4,1,1494,0.001937,158,0.550633,1549,0.678502,1.0


# Fix 0. `product_user_coeff`
Поскульку надо разделить эмбеддинги, мы уберём эту фичу. Кроме того она неверно посчитана.

In [53]:
df.drop(columns=["product_user_coeff"], inplace=True)

In [56]:
df.head()

,product_id,user_id,user_total_bought,user_reorder_coeff,product_total_ordered,product_reorder_coeff,reordered_target
0,1,138,148,0.628378,1549,0.678502,1.0
1,1,777,113,0.734513,1549,0.678502,0.0
2,1,1052,48,0.500000,1549,0.678502,1.0
3,1,1480,192,0.734375,1549,0.678502,1.0
4,1,1494,158,0.550633,1549,0.678502,1.0


# Fix 1. Номализация

давайте обсудим нейросеть: 

Достаточно использовать только dense слои

модель должна принимать на вход эмбеддинги пользователя + эмбеддинги какого то продукта, на выходе получаем вероятность покупки этого продукта 

После для каждого пользователя сортируем все вероятности по убыванию и берем первые 10 

Активация на выходе - сигмоид 

In [54]:
import warnings
warnings.filterwarnings("ignore")
import torch
import numpy as np
from sklearn.model_selection import train_test_split

# Fix 2. `user_id` и `product_id` включены в датасет

In [62]:
Xp_cols = ["product_id", "product_total_ordered", "product_reorder_coeff"]

In [63]:
Xu_cols = ["user_id", "user_total_bought", "user_reorder_coeff"]

In [64]:
y_cols = ["reordered_target"]

# Fix 3. Эмбеддинги разделены на Xp и Xu

In [65]:
Xp = df[Xp_cols]
Xu = df[Xu_cols]
y = df[y_cols]

In [66]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [67]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(5, 120)
        self.fc2 = nn.Linear(120, 120)
        self.fc3 = nn.Linear(120, 120)
        self.fc4 = nn.Linear(120, 120)
        self.fc5 = nn.Linear(120, 120)
        self.fc6 = nn.Linear(120, 84)
        self.fc7 = nn.Linear(84, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = F.sigmoid(self.fc7(x))
        return x

In [68]:
from torch.utils.data import Dataset, DataLoader

# Convert data to torch tensors
class Data(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.float32))
        self.len = self.X.shape[0]
       
    def __getitem__(self, index):
        return self.X[index], self.y[index]
   
    def __len__(self):
        return self.len

In [69]:
model = Net()
loss_fn = nn.BCELoss()

In [70]:
model = model.to("cuda")

In [71]:
batch_size = 2048

# Instantiate training and test data
train_data = Data(X.values, y.values)
train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

# test_data = Data(X_test.values, y_test.values)
# test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)

NameError: name 'X' is not defined

In [72]:
from tqdm import tqdm
import torch.optim as optim

# Fix 4. Оптимизатор заменён на Adam

In [76]:
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [35]:
num_epochs = 300
loss_values = []


for epoch in tqdm(range(num_epochs)):
    for X, y in train_dataloader:
        X = X.to("cuda")
        y = y.to("cuda")
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        pred = model(X)
        loss = loss_fn(pred, y.unsqueeze(-1)[:,:,0])
        loss_values.append(loss.item())
        loss.backward()
        optimizer.step()

print("Training Complete")

100%|██████████| 300/300 [6:59:10<00:00, 83.83s/it]   

Training Complete


In [36]:
model = model.to("cpu")

In [ ]:
predictions = []

for X, y in tqdm(train_dataloader):
    pred = model(X)
    predictions.append(pred.detach().numpy())

 31%|███       | 1442/4619 [00:31<01:02, 50.60it/s]

In [ ]:
df["predictions"] = np.vstack(predictions)

In [39]:
df.to_csv("predictions.csv")

In [45]:
ranked_lists = df.sort_values(["user_id", "predictions"], ascending=False).groupby("user_id")["product_id"].agg(list)

top 10 sales

In [60]:
top_20 = list(transactions.groupby("product_id")["user_id"].agg("count").sort_values()[-20:].sort_values(ascending=False).keys())

In [73]:
def fill_10(row):
    if len(row) == 10:
        return row.product_id
    pids = row.product_id
    diff = set(top_20) - set(pids)
    unique_top_ranged = [p for p in top_20 if p in diff]
    res = pids + unique_top_ranged
    return res[:10]

In [89]:
ranked_lists = ranked_lists.reset_index()

predictions_list = ranked_lists.apply(fill_10, axis=1)

ranked_lists.product_id = predictions_list

ranked_lists.product_id = ranked_lists.product_id.apply(lambda x: " ".join(map(str, x)))

ranked_lists.columns = ["user_id", "product_id"]

ranked_lists.to_csv("sample_submission_2.csv", index=False)